In [35]:
# RIKER 1.0.0 - Facial Data is imported (after having been processed by OpenFace with -aus flag),
# and data is temporally segmented to identify the start, end, and peak of individual 
# Facial Action units. Then, individual actions are aggregated together into a DataFrame,
# Then, the aggregated actions are clustered together (using DBSCAN algorithm)
# into Facial Gestures, and the FGs are filtered (very minor gestures are removed).
# Lastly, timestamp information is loaded in from the .log file, and FGs are matched up with the
# trimmed to include only FGs that ocurred during the poker game (those that ocurred before and 
# after are discarded), and the FGs are given new timestamps relative to the beginning
# of the poker game.

import time
from datetime import datetime, timedelta
# import datetime
import warnings

#from ipynb.fs.full.Game_State_Parser import *

import numpy as np
import matplotlib.pyplot as pp
# import seaborn
# from collections import deque

from sklearn import cluster, datasets, mixture
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice, chain
# from sklearn.cluster import AffinityPropagation
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.metrics import mean_squared_error

import pandas as pd

%matplotlib inline

In [36]:
#Reads a log file of time stamps that indicate when the PokerTH log file was modified (i.e. when an action was taken in the game).
def get_timestamps(ts_filename_):

    timestamps = []

    with open(ts_filename_) as f:
        for line in f:
            if "Modified file" in line:
                for word in line.split():
                    if "," in word:
                        #print(word)
                        timestamps.append(word)
                        
        #Extra lines added for overflow in timestamps matching with PokerActs later...
        timestamps.append("00:00:00,000")
        timestamps.append("00:00:00,000")
        timestamps.append("00:00:00,000")
        timestamps.append("00:00:00,000")

    return(timestamps)

In [37]:
#Reads in the data from a .csv file as output by OpenFace FeatureExtraction with the '-aus' flag on.
def parse_csv(filename):
    load_cols = [0] + [i for i in range (2,22)]                              #which columns we want to use
    load_names = ['frame','timestamp','confidence','success','AU01','AU02',  #names of the columns
            'AU04','AU05','AU06','AU07','AU09','AU10','AU12','AU14',        
            'AU15','AU17','AU20','AU23','AU25','AU26','AU45']
    load_types = [np.int32,np.float16,np.float32,np.int32] + [np.float32]*17 #types of the columns

    load_file = np.genfromtxt(fname = filename,
                  usecols = load_cols,
                  dtype = load_types,
                  skip_header = 1,
                  delimiter = ',',
                  names = load_names)
    return load_file

In [38]:
def plot_raw_data(rawdata):
    pp.figure(figsize=(25,10))

    pp.plot(rawdata['frame'],rawdata['AU01'])#, label='AU01')
    pp.plot(rawdata['frame'],rawdata['AU02'])#, label='AU02')
    pp.plot(rawdata['frame'],rawdata['AU04'])#, label='AU04')
    pp.plot(rawdata['frame'],rawdata['AU05'])#, label='AU05')
    pp.plot(rawdata['frame'],rawdata['AU06'])
    pp.plot(rawdata['frame'],rawdata['AU07'])#, label='AU07')
    pp.plot(rawdata['frame'],rawdata['AU09'])
    pp.plot(rawdata['frame'],rawdata['AU10'])
    pp.plot(rawdata['frame'],rawdata['AU12'])
    pp.plot(rawdata['frame'],rawdata['AU14'])
    pp.plot(rawdata['frame'],rawdata['AU15'])
    pp.plot(rawdata['frame'],rawdata['AU17'])
    pp.plot(rawdata['frame'],rawdata['AU20'])#, label='AU20')
    pp.plot(rawdata['frame'],rawdata['AU23'])
    pp.plot(rawdata['frame'],rawdata['AU25'])
    pp.plot(rawdata['frame'],rawdata['AU26'])#, label='AU26')
    pp.plot(rawdata['frame'],rawdata['AU45'])
    pp.legend()

In [39]:
def data_smoothed(t, win=5):
    AU01 = np.correlate(t['AU01'],np.ones(win)/win,'same')
    AU02 = np.correlate(t['AU02'],np.ones(win)/win,'same')
    AU04 = np.correlate(t['AU04'],np.ones(win)/win,'same')
    AU05 = np.correlate(t['AU05'],np.ones(win)/win,'same')
    AU06 = np.correlate(t['AU06'],np.ones(win)/win,'same')
    AU07 = np.correlate(t['AU07'],np.ones(win)/win,'same')
    AU09 = np.correlate(t['AU09'],np.ones(win)/win,'same')
    AU10 = np.correlate(t['AU10'],np.ones(win)/win,'same')
    AU12 = np.correlate(t['AU12'],np.ones(win)/win,'same')
    AU14 = np.correlate(t['AU14'],np.ones(win)/win,'same')
    AU15 = np.correlate(t['AU15'],np.ones(win)/win,'same')
    AU17 = np.correlate(t['AU17'],np.ones(win)/win,'same')
    AU20 = np.correlate(t['AU20'],np.ones(win)/win,'same')
    AU23 = np.correlate(t['AU23'],np.ones(win)/win,'same')
    AU25 = np.correlate(t['AU25'],np.ones(win)/win,'same')
    AU26 = np.correlate(t['AU26'],np.ones(win)/win,'same')
    AU45 = np.correlate(t['AU45'],np.ones(win)/win,'same')
    
    smoothedAUs = np.vstack((AU01, AU02, AU04, AU05, AU06, AU07, AU09, AU10, AU12, AU14, AU15, AU17, AU20, AU23, AU25, AU26, AU45))
    
    return(smoothedAUs)

In [40]:
def au_detect(action_unit,label):
    # ***COMMENT NEEDS UPDATE***An array containing codes for noting when gestures started(1), peaked(2), and ended(3) during the sequence, as
    # well as the amplitude for the that gesture (encoded as the amplitude change from start to peak), and the 
    # duration of the gesture (encoded as the distance between the start and the end indices).
    
    au_data = np.array([0,0,0,0,0]).reshape((1,5))
    au_df = pd.DataFrame(au_data, columns=list('SPEBT'))
    au_df['AU Label'] = [label]
    #print(au_df)
    
    begun = False
    rising = False
    peaked = False
    ending = False
    sensitivity = 0.07
    start_index = 0
    start_amp = 0
    peak_index = 0
    peak_amp = 0
    
    
    for i in range(len(action_unit)):
        if i > 1:
            if action_unit[i] - action_unit[i-2] >= sensitivity and rising == False:
                rising = True
                begun = True
                output = "Started a gesture at " + repr(i)
                start_amp = action_unit[i-2]
                next_au = np.array([start_index,peak_index,i-1,start_amp,peak_amp]).reshape((1,5))
                next_au_df = pd.DataFrame(next_au, columns=list('SPEBT'))
                next_au_df['AU Label'] = [label]
                new_list = au_df, next_au_df
                au_df = pd.concat(new_list)
                start_index = i
                peak_index = 0
            elif action_unit[i] - action_unit[i-2] >= sensitivity and rising == True:
                output = "Kept going up at " + repr(i)
            elif action_unit[i] + action_unit[i-1] < sensitivity and rising == False and ending == False:
                output = "Still at baseline at " +repr(i)
            elif action_unit[i] - action_unit[i-1] <= sensitivity and rising == True:
                rising = False
                peaked = True
                ending = False
                output = "Peaked at " + repr(i)
                peak_index = i
                peak_amp = action_unit[i-1]
            elif action_unit[i] - action_unit[i-2] <= (0 - sensitivity) and rising == False and peaked == True:
                rising = False
                peaked = True
                ending = True
                output = "Coming down at " + repr(i-1)
            elif action_unit[i] - action_unit[i-2] < sensitivity and ending == True and action_unit[i] > sensitivity:
                output = "Held steady for now at " + repr(i-1)
            elif action_unit[i] - action_unit[i-2] < sensitivity and ending == True and action_unit[i] < sensitivity:
                begun = False
                rising = False
                peaked = False
                ending = False
                output = "Back to baseline at " + repr(i)
                next_au = np.array([start_index,peak_index,i-1,start_amp,peak_amp]).reshape((1,5))
                next_au_df = pd.DataFrame(next_au, columns=list('SPEBT'))
                next_au_df['AU Label'] = [label]
                new_list = au_df, next_au_df
                au_df = pd.concat(new_list)
                start_index = i
                peak_index = i
                peak_amp = 0
                     
    return(au_df)

In [41]:
#Combine the facial actions into a single dataframe.
def aggregate_actions(smoothed_actions):
    AU_labels = ['01','02','04','05','06','07','09','10','12','14','15','17','20','23','25','26','45']

    action_data = np.array([0,0,0,0,0]).reshape((1,5))
    action_df = pd.DataFrame(action_data, columns = ['S','P','E','B','T']) #Start Time, Peak Time, End Time, Bottom Amp, Top Amp
    action_df['AU Label'] = [0]
    action_df['Index'] = 0

    for i in range(len(smoothed_actions)):
        all_actions = au_detect(smoothed_actions[i],AU_labels[i])
        new_list = action_df, all_actions
        action_df = pd.concat(new_list)
    
    #Remove actions which End at frame 0.
    action_df = action_df[action_df.E != 0]
    
    #Add the index column
    index_list = list(range(0,len(action_df['Index'])))
    for i in range(len(action_df['Index'])):
        action_df.iloc[i,3:4] = index_list[i]
    action_df.set_index('Index',inplace=True)
    
    #Add the TotalAmp column
    action_df['TotalAmp'] = 0

    for i in range(len(action_df)):
        float_index = float(i)
        action_df.iloc[i,6:7] = action_df.at[float_index,'T'] - action_df.at[float_index,'B']
        
    #Add a column containing the timing of the Peak in seconds
    action_df['Inflection'] = 0
    for i in range(len(action_df)):
        float_index = float(i)
        inflection_point = (action_df.at[float_index,'P'])
        action_df.iloc[i,7:8] = inflection_point/30
        
    #Add a column containing the duration of action onset
    action_df['Onset Frame Count'] = 0
    for i in range(len(action_df)):
        float_index = float(i)
        onset_length = (action_df.at[float_index,'P'])-(action_df.at[float_index,'S'])
        action_df.iloc[i,8:9] = onset_length
    
    #Add a column containing the duration of the action offset
    action_df['Offset Frame Count'] = 0
    for i in range(len(action_df)):
        float_index = float(i)
        offset_length = (action_df.at[float_index,'E'])-(action_df.at[float_index,'P'])
        action_df.iloc[i,9:10] = offset_length
    
    #Drop the actions with a TotalAmp < 0
    action_df['Drop'] = 0
    for i in range(len(action_df)):
        float_index = float(i)
        if action_df.at[float_index,'TotalAmp'] < 0.01:
            action_df.at[float_index,'Drop'] = 1
    action_df = action_df[action_df.Drop != 1]
    
    #Sort by peaks of data
    peaks_sort = action_df.sort_values(['P','AU Label'], ascending=[True,True])
    
    return(peaks_sort)

In [42]:
#Adds the Gesture label to a dataframe of facial actions, so we can tell which gesture each action belongs to.
def add_gestures(actions_df,gesture_group_labels):
    actions_df['Gesture'] = 0

    for i in range(len(gesture_group_labels)):
        actions_df.iloc[i,11:12] = gesture_group_labels[i]

    return(actions_df)

In [43]:
#Creates a dataframe of gestures which is built from a dataframe of actions.
def actions_to_gestures(actions_df_in,n_action_clusters_in):
    #Setting up new dataframe to receive FGs and metadata
    all_gestures_df = pd.DataFrame(index=range(0,n_action_clusters_in), columns = ['01','02','04','05','06','07','09','10','12','14','15','17','20','23','25','26','45'], dtype='float')
    all_gestures_df['Inflection'] = 0
    all_gestures_df['Onset Length'] = 0 #Mean number of frames of action onsets
    all_gestures_df['Onset Unity'] = 0 #Variance in frames of action onsets
    all_gestures_df['Offset Length'] = 0 #Mean number of rames of action offsets
    all_gestures_df['Offset Unity'] = 0 #Variance in frames of action offsets

    # For each Gesture
    for i in range(n_action_clusters_in):     
        gesture_df = actions_df_in[actions_df_in.Gesture == i].set_index('AU Label')

        #Add the mean inflection point(peak point) for the gesture to the df
        inflection_point = gesture_df['Inflection'].mean()
        #print("Inflection point is " + repr(inflection_point))
        all_gestures_df.iloc[i,17:18] = inflection_point
        
        #Add the onset length
        onset_length = gesture_df['Onset Frame Count'].mean()
        all_gestures_df.iloc[i,18:19] = onset_length
        
        #Add the onset unity
        onset_unity = gesture_df['Onset Frame Count'].var()
        all_gestures_df.iloc[i,19:20] = onset_unity
        
        #Add the offset length
        offset_length = gesture_df['Offset Frame Count'].mean()
        all_gestures_df.iloc[i,20:21] = offset_length
        
        #Add the onset unity
        offset_unity = gesture_df['Offset Frame Count'].var()
        all_gestures_df.iloc[i,21:22] = offset_unity
        
        #For each action unit observed
        for j in range(len(gesture_df)):
            #For each column in all_gestures_df, except 'Inflection'
            for k in range(len(all_gestures_df.columns)-1): 
                pos = k
                colname = all_gestures_df.columns[pos]
                if colname in gesture_df.index:
                    all_gestures_df.at[i,colname] = gesture_df.iloc[0,5:6]
            if len(gesture_df) > 1:
                gesture_df = gesture_df.iloc[1:]
    
    #Fill in missing values with 0s
    all_gestures_filled = all_gestures_df.fillna(0)    
    
    #Total Amp of all Actions in the Gesture
    all_gestures_filled['SumAmp'] = all_gestures_filled.sum(axis=1)
    
    #Fix the SumpAmp so it doesn't include the Inflection Point (and other metadata) in the sum
    for i in range(len(all_gestures_filled)):
        all_gestures_filled.iloc[i,22:23] = all_gestures_filled.iat[i,22] - all_gestures_filled.iat[i,21] - all_gestures_filled.iat[i,20] - all_gestures_filled.iat[i,19] - all_gestures_filled.iat[i,18] - all_gestures_filled.iat[i,17]

    return(all_gestures_filled)

In [44]:
#Removes gestures from a gestures dataframe which have very small amplitudes.
def gesture_filter_low(gestures_df):
    gestures_df['Drop'] = 0
    
    # Identify which gestures have SumAmps < 0.2...
    for i in range(len(gestures_df)):
        if gestures_df.at[i,'SumAmp'] < 0.2:
            gestures_df.iloc[i,23:24] = 1
    
    # ...and drop them.
    gestures_df = gestures_df[gestures_df.Drop != 1]
    
    #Reset the Index so it is still continuous
    gestures_df.reset_index(inplace=True)
    gestures_df = gestures_df.loc[:,'01':'SumAmp']

    return(gestures_df)

In [45]:
#Identifies the AUs that start and peak at similar times, and clusters them together.
def DBSCAN_propagate(all_smoothed_AUs):
    
    # Gather all the Facial Actions into a single dataframe, sort that data frame by when the actions
    # peak in amplitude.
    sort_by_peaks = aggregate_actions(all_smoothed_AUs)
    peaks_sort = sort_by_peaks.sort_values(['P','AU Label'], ascending=[True,True])
    
    #Only count actions with a peak amplitude of greater than 0. 
    X = peaks_sort.loc[peaks_sort['P']>=0.0,['P','S']].values
    
    # Compute DBSCAN
    clustering = DBSCAN(eps=6.7, min_samples=2, n_jobs=-1).fit(X)
    cluster_centers_indices = clustering.core_sample_indices_
    labels = clustering.labels_
    n_clusters_ = len(cluster_centers_indices)
    
    # Plot clusters: UNCOMMENT THE FOLLOWING LINES UNTIL THE 'RETURN' LINE
    # IN ORDER TO GET A VISUAL GRAPHIC OF THE CLUSTERS. NOTE: THIS GRAPHIC
    # WILL LIKELY NOT BE VERY USEFUL FOR MORE THAN A FEW HUNDRED FRAMES OF
    # SUBJECT DATA – ANYTHING LONGER THAN THAT WILL JUST APPEAR TOO SMALL.
    # THIS IS BEST USED TO VALIDATE THAT THE FUNCTION IS WORKING PROPERLY
    # ON A SMALL/SHORT EXAMPLE DATASET.
    
#     core_samples_mask = np.zeros_like(clustering.labels_, dtype=bool)
#     core_samples_mask[clustering.core_sample_indices_] = True
    
    
#     pp.close('all')
#     pp.figure(figsize=(13,12))
#    # Black removed and is used for noise instead.
#     unique_labels = set(labels)
#     colors = [pp.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
#     for k, col in zip(unique_labels, colors):
#         if k == -1:
#             # Black used for noise.
#             col = [0, 0, 0, 1]

#         class_member_mask = (labels == k)

#         xy = X[class_member_mask & core_samples_mask]
#         pp.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
#                  markeredgecolor='k', markersize=3)

#         xy = X[class_member_mask & ~core_samples_mask]
#         pp.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
#                  markeredgecolor='k', markersize=1)

#     pp.title('Estimated number of clusters: %d' % n_clusters_)
#     pp.show()
    
    return(labels,n_clusters_)

In [46]:
def get_FGs(subject_num_):
    raw_data = parse_csv(str(subject_num_) + '_FaceOnly.csv')
    # #plot_raw_data(raw_data)
    print("Data is imported")
    all_AUs_smoothed = data_smoothed(raw_data)
    print("Data is smoothed")
    actions_df = aggregate_actions(all_AUs_smoothed)
    print("Actions are aggregated into dataframe")
    # #print(actions_df)
    gesture_group_labels, n_clusters = DBSCAN_propagate(all_AUs_smoothed)
    # gesture_group_labels
    print("Gestures/clusters are identified")
#     print("GESTURE GROUP LABELS ARE okay")
#     # #print(gesture_group_labels)
#     print("n_clusters is okay")
    # #print(n_clusters)
    act_with_gest = add_gestures(actions_df,gesture_group_labels)
    print("And actions have gesture labels.")
    # #print(act_with_gest)
    gestures_df = actions_to_gestures(act_with_gest,n_clusters)
    print("Gestures are in a data frame.")
    # #print(gestures_df)
    filtered_gestures_ = gesture_filter_low(gestures_df)
    print("Gestures are low-pass filtered.")
    
    return(filtered_gestures_)

In [51]:
def trim_gestures_to_game(filtered_gestures_, times_, rec_start_time_):
    print("Trimming to fit game start and end time stamps.")
    game_start_time = datetime.strptime(times_[0], "%H:%M:%S,%f")
    game_end_time = datetime.strptime(times_[-6], "%H:%M:%S,%f")

    start_diff = game_start_time - rec_start_time_
    end_diff = game_end_time - rec_start_time_
    secs = start_diff.seconds
    mils = start_diff.microseconds
    end_secs = end_diff.seconds
    end_mils = end_diff.microseconds

    relative_start = float(secs+(mils/1000000))
    relative_end = float(end_secs + (end_mils/1000000))

    game_gests_ = filtered_gestures_.loc[filtered_gestures_['Inflection'] > relative_start]
    game_gests_ = game_gests_.loc[filtered_gestures_['Inflection'] < (relative_end + 10)]
    # game_gests = 
    game_gests_['TrueInflection'] = 0
    for i in range(0,len(game_gests_)):
        game_gests_.iloc[i,-1] = game_start_time + timedelta(seconds = (game_gests_.iloc[i,-7]-relative_start))
        
    return(game_gests_)

In [50]:
#Main execution of program components

#Subject-by-subject metadata, including "subject number" and start time of their game.
subject_map = pd.read_csv('subject_map.csv')

# For each subject on which we have data – with example of just subject 1,
# keep this in range 1,2.
for subject_num in range(1,2):
    print("STARTING SUBJECT NUMBER " + repr(subject_num))
    
    # Calculate Facial Gestures (FGs) from facial data as processed by OpenFace.
    filtered_gestures = get_FGs(subject_num)
    
    #Get time stamps from subject log
    ts_filename = str(subject_num) + "_timestamps.log"
    times = get_timestamps(ts_filename)
    rec_start_time = datetime.strptime(subject_map.iloc[subject_num-1,1], "%H:%M:%S,%f")
    
    # Trim FGs to be FGs that ocurred during the poker game
    game_gests = trim_gestures_to_game(filtered_gestures, times, rec_start_time)

    #UNCOMMENT THIS NEXT LINE TO OUTPUT THE RESULTS TO THE NEXT PHASE
    game_gests.to_csv(str(subject_num)+ "TEST_FGs_withcorrecttimestamps.csv")

STARTING SUBJECT NUMBER 1
Data is imported
Data is smoothed
Actions are aggregated into dataframe
Gestures/clusters identified
And actions have gesture labels.
Gestures are in a data frame.
Gestures are low-pass filtered.
